In [ ]:
!pip install rank_bm25 datasets ranx scikit-learn -q

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.2/249.2 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.0/859.0 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.2/119.2 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from datasets import load_dataset
import pandas as pd
from rank_bm25 import BM25Okapi
from ranx import Qrels, Run, evaluate
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
dataset = load_dataset("ShoAnn/legalqa_klinik_hukumonline")
print("Dataset loaded:")
print(dataset)

README.md:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.62M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/547k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1006 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/112 [00:00<?, ? examples/s]

Dataset loaded:
DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'context'],
        num_rows: 1006
    })
    test: Dataset({
        features: ['question', 'answer', 'context'],
        num_rows: 112
    })
})


In [ ]:
corpus = []
doc_id_map = {} # Map original ID to corpus index
corpus_docs_original_ids = [] # Store original IDs in corpus order

doc_index = 0
for docs in dataset['train']['context']+dataset['test']['context']:
    for doc in docs:
        full_text = "".join([char for char in doc['full_text'].strip() if char not in ["'"]])
        corpus_entry = f"{doc['name'].strip()} : '{full_text}'"
        corpus.append(corpus_entry)

        original_id = str(doc['id'])
        doc_id_map[original_id] = str(doc_index)
        corpus_docs_original_ids.append(original_id)

        doc_index += 1

doc_ids_for_retrieval = [str(i) for i in range(len(corpus))]

test_queries = {}
for i in range(len(dataset['test']['question'])):
    test_queries[f"q{i}"] = dataset['test']['question'][i]
print(f"\nPrepared {len(test_queries)} test queries.")
print(f"Sample query (q0): {test_queries['q0']}")


Prepared 112 test queries.
Sample query (q0): Saya menggunakan GPS ketika berkendara, kata temen saya itu bisa dipidana. Apakah benar?


In [ ]:
qrels_dict = {}
count_relevant_docs_found = 0
count_total_relevant_docs = 0

for i in range(len(dataset['test'])):
    query_key = f"q{i}"
    qrels_dict[query_key] = {}
    rel_docs_original_ids = [str(doc['id']) for doc in dataset['test'][i]['context']]
    count_total_relevant_docs += len(rel_docs_original_ids)

    for original_id in rel_docs_original_ids:
        if original_id in doc_id_map:
            corpus_index_id = doc_id_map[original_id]
            # relevance score 0 1 (binary relevance)
            qrels_dict[query_key][corpus_index_id] = 1
            count_relevant_docs_found += 1

print(f"Created Qrels for {len(qrels_dict)} queries.")
print(f"Total relevant documents listed in test set: {count_total_relevant_docs}")
print(f"Relevant documents found in corpus: {count_relevant_docs_found}")

Created Qrels for 112 queries.
Total relevant documents listed in test set: 772
Relevant documents found in corpus: 772


In [ ]:
print("Starting BM25 indexing...")
tokenized_corpus_bm25 = [doc.lower().split() for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus_bm25)
print("BM25 index built.")

print("\nStarting BM25 retrieval...")
bm25_run_dict = {}
for q_id, query_text in test_queries.items():
    tokenized_query = query_text.lower().split()
    doc_scores = bm25.get_scores(tokenized_query)

    # Create list of (doc_id, score) tuples and sort descending by score
    scored_docs = sorted(zip(doc_ids_for_retrieval, doc_scores), key=lambda x: x[1], reverse=True)

    bm25_run_dict[q_id] = {doc_id: score for doc_id, score in scored_docs}
print("BM25 retrieval finished.")
print("BM25 Run Dict Sample (q0):", dict(list(bm25_run_dict["q0"].items())[:5]))

Starting BM25 indexing...
BM25 index built.

Starting BM25 retrieval...
BM25 retrieval finished.
BM25 Run Dict Sample (q0): {'6158': np.float64(23.2844609625998), '2021': np.float64(13.485694537116581), '5018': np.float64(13.388843949307596), '1813': np.float64(13.225246218635968), '3073': np.float64(13.073313916870976)}


In [ ]:
print("Starting TF-IDF vectorization (indexing)...")
# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(lowercase=True)

tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)
print(f"TF-IDF matrix shape: {tfidf_matrix.shape}")
print("TF-IDF indexing finished.")

print("\nStarting TF-IDF retrieval...")
tfidf_run_dict = {}
for q_id, query_text in test_queries.items():
    query_vector = tfidf_vectorizer.transform([query_text])

    # Calculate cosine similarity between the query vector and all document vectors
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten() # [[sim_q_d1, sim_q_d2, ...]]

    # Create list of (doc_id, score) tuples and sort descending by score
    scored_docs_tfidf = sorted(zip(doc_ids_for_retrieval, cosine_similarities), key=lambda x: x[1], reverse=True)

    tfidf_run_dict[q_id] = {doc_id: score for doc_id, score in scored_docs_tfidf}

print("TF-IDF retrieval finished.")
print("TF-IDF Run Dict Sample (q0):", dict(list(tfidf_run_dict["q0"].items())[:5]))

Starting TF-IDF vectorization (indexing)...
TF-IDF matrix shape: (7037, 13324)
TF-IDF indexing finished.

Starting TF-IDF retrieval...
TF-IDF retrieval finished.
TF-IDF Run Dict Sample (q0): {'6158': np.float64(0.1427313748724156), '1977': np.float64(0.12377044304078524), '1978': np.float64(0.12377044304078524), '2504': np.float64(0.12377044304078524), '5212': np.float64(0.12377044304078524)}


In [ ]:
qrels = Qrels(qrels_dict)
bm25_run = Run(bm25_run_dict)
tfidf_run = Run(tfidf_run_dict)

metrics = ["recall@5", "recall@10", "recall@20", "recall@100", "recall@1000",
           "mrr@5", "mrr@10", "mrr@20", "mrr@100", "mrr@1000"]

print("Evaluating BM25...")
bm25_results = evaluate(qrels, bm25_run, metrics)
print("\nBM25 Evaluation Results (Raw Dictionary):")
print(bm25_results)

print("\nEvaluating TF-IDF...")
tfidf_results = evaluate(qrels, tfidf_run, metrics)
print("\nTF-IDF Evaluation Results (Raw Dictionary):")
print(tfidf_results)

Evaluating BM25...


/usr/local/lib/python3.11/dist-packages/ranx/metrics/recall.py:29: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  scores[i] = _recall(qrels[i], run[i], k, rel_lvl)



BM25 Evaluation Results (Raw Dictionary):
{'recall@5': np.float64(0.07585629385786949), 'recall@10': np.float64(0.10048356755289527), 'recall@20': np.float64(0.1336151097968325), 'recall@100': np.float64(0.2437847574927407), 'recall@1000': np.float64(0.4689330299427463), 'mrr@5': np.float64(0.15907738095238094), 'mrr@10': np.float64(0.17068098072562357), 'mrr@20': np.float64(0.17490726404198043), 'mrr@100': np.float64(0.1794321145156805), 'mrr@1000': np.float64(0.18024097534703104)}

Evaluating TF-IDF...

TF-IDF Evaluation Results (Raw Dictionary):
{'recall@5': np.float64(0.08206163001331068), 'recall@10': np.float64(0.13444368496232992), 'recall@20': np.float64(0.18602029899955322), 'recall@100': np.float64(0.33007001023019933), 'recall@1000': np.float64(0.550376213276686), 'mrr@5': np.float64(0.2074404761904762), 'mrr@10': np.float64(0.2248689058956916), 'mrr@20': np.float64(0.23177762582305647), 'mrr@100': np.float64(0.2344493418588081), 'mrr@1000': np.float64(0.2352130457086461)}


In [ ]:
print("--- BM25 Results ---")
for metric, value in bm25_results.items():
    print(f"{metric}: {float(value):.4f}") # Format to 4 decimal places

print("\n--- TF-IDF Results ---")
for metric, value in tfidf_results.items():
    print(f"{metric}: {float(value):.4f}")

--- BM25 Results ---
recall@5: 0.0759
recall@10: 0.1005
recall@20: 0.1336
recall@100: 0.2438
recall@1000: 0.4689
mrr@5: 0.1591
mrr@10: 0.1707
mrr@20: 0.1749
mrr@100: 0.1794
mrr@1000: 0.1802

--- TF-IDF Results ---
recall@5: 0.0821
recall@10: 0.1344
recall@20: 0.1860
recall@100: 0.3301
recall@1000: 0.5504
mrr@5: 0.2074
mrr@10: 0.2249
mrr@20: 0.2318
mrr@100: 0.2344
mrr@1000: 0.2352
